# Type assertion

**Type assertion** - это операция, которую можно применить к значению интерфейсного типа. В общем виде эта операция выглядит как **x.(T)**, где x - это выражение интерфейсного типа, а T - некоторый тип(конкретный или интерфейсный). Type assertion работает с динамическим типом интерфейсного значения. Тут возможны два варианта:  
1. **T является конкретным типом**. В этом случае проверяется совпадает ли динамический тип x с типом T. Если совпадает, то результатом является динамическое значение x, которое конечно же имеет тип T. Другими словами, операция type assertion с конкретным типом извлекает конкретное значение из операнда:

In [1]:
import (
    "fmt"
    "io"
    "os"
)

var w io.Writer      // Интерфейсное значение.
w = os.Stdout        // Присваивание проходит, т.к. os.Stdout удовлетворяет интерфейсу io.Writer.
f := w.(*os.File)    // Type assertion проходит, т.к. динамический тип w это тип *os.File.
fmt.Sprintf("%T", f)

*os.File


 Если динамический тип x не равен T, то происходит **паника**:

In [ ]:
import "bytes"
// Следующий код вызовет панику.
c := w.(*bytes.Buffer)

2. **Т является интерфейсным типом**. В этом случае проверяется удовлетворяет ли динамический тип x интерфейсному типу T. Если удовлетворяет, то результатом операции является значение интерфейсного типа T, у которого динамические тип и значение точно такие же, как и у интерфейсного значения x. Другими словами, type assertion с интерфейсным типом изменяет тип выражения, возможно позволяя получить доступ к большему количеству методов.